# Plot data from the VIEWS API, CM level

In [1]:
# TODO: Needs refactoring and incorporating automatic production of pgm maps for the same data selection. PGM is currently a separate file. 

In [2]:
import requests
import pandas as pd
import json
import os
from pathlib import Path

#import matplotlib.pyplot as plt

# External
import geopandas as gpd
import sqlalchemy as sa

# VIEWS
from viewser.operations import fetch
from views_mapper2.mapper2 import *
from views_mapper2.BBoxWriter import *
from views_mapper2.dictionary_writer import *
from ingester3.config import source_db_path

## Select forecasts to fetch

In [3]:
# List the dataset specified in the URL above

model = 'fatalities002'
release = '2024_06_t01'
dataset = f'{model}_{release}'

# Define the URL you want to fetch data from. 

loa = 'cm'

FORECASTS = f'https://api.viewsforecasting.org/{dataset}/{loa}'
FORECASTS 


'https://api.viewsforecasting.org/fatalities002_2024_06_t01/cm'

## Set prediction step (month) to analyze

In [4]:
step = 12 # set step to plot

## Fetch forecasts from VIEWS API

In [5]:
# Request function
r = requests.get(FORECASTS)

#The set page size 
PAGE_SIZE=1000

#Define the full request 
req_url=FORECASTS #+'/'+str()+'?'+'pagesize='+str(PAGE_SIZE)
    
master_list=[]
r=requests.get(req_url)
page_data=r.json()

list(page_data)

['next_page',
 'prev_page',
 'model_tree',
 'models',
 'row_count',
 'page_count',
 'page_cur',
 'start_date',
 'end_date',
 'data']

In [6]:
# The loop to keep fetching pages until all data is accessed 

master_list+=page_data['data']
    
while page_data['next_page'] != '':
    r=requests.get(page_data['next_page'])
    page_data=r.json()

    master_list+=page_data['data']
        
forecasts=pd.DataFrame(master_list)
forecasts

,country_id,month_id,name,gwcode,isoab,year,month,surrogate_mean_dem_ln,surrogate_mean_imr,surrogate_mean_pop_ln,surrogate_mean_nch_ln,surrogate_mean_ch,surrogate_mean_dem,surrogate_mean_pop,main_dich,surrogate_mean_imr_ln,main_mean_ln,surrogate_mean_ch_ln,main_mean,surrogate_mean_nch
0,1,535,Guyana,110,GUY,2024,7,0.8658,0.7493,0.1155,0.3121,0.0777,1.3769,0.1225,0.0026,0.5592,0.0085,0.0748,0.0085,0.3663
1,2,535,Suriname,115,SUR,2024,7,0.1352,0.5163,0.1097,0.3121,0.0777,0.1448,0.1160,0.0026,0.4163,0.0108,0.0748,0.0109,0.3663
2,3,535,Trinidad and Tobago,52,TTO,2024,7,0.1334,0.4924,0.1373,0.1040,0.0777,0.1427,0.1472,0.0026,0.4004,0.0112,0.0748,0.0113,0.1095
3,4,535,Venezuela,101,VEN,2024,7,0.8688,0.6853,0.8416,1.0711,0.0777,1.3839,1.3200,0.0031,0.5220,0.1070,0.0748,0.1129,1.9186
4,5,535,Samoa,990,WSM,2024,7,0.2235,0.4864,0.0978,0.1040,0.0777,0.2504,0.1027,0.0027,0.3964,0.0193,0.0748,0.0194,0.1095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6871,242,570,Tanzania,510,TZA,2027,6,0.7498,1.4461,1.7149,2.8005,0.2581,1.1165,4.5562,0.0170,0.8945,0.7024,0.2296,1.0187,15.4535
6872,243,570,Morocco,600,MAR,2027,6,1.2241,0.7731,1.2023,0.4784,0.2581,2.4012,2.3276,0.0109,0.5728,0.3781,0.2296,0.4594,0.6135
6873,244,570,Mauritania,435,MRT,2027,6,1.4308,1.3701,0.3507,1.1283,0.2581,3.1819,0.4201,0.0134,0.8629,0.5324,0.2296,0.7030,2.0905
6874,245,570,Sudan,625,SDN,2027,6,1.0247,1.6683,1.3856,2.8159,202.5655,1.7861,2.9971,0.9497,0.9814,5.7026,5.3160,298.6518,15.7081


In [7]:
model

'fatalities002'

In [8]:
# Filter out the relevant data columns, and rename them where needed

if model == 'fatalities001':
        columns_to_keep=['country_id','month_id','sc_cm_sb_main']
        forecasts = forecasts[columns_to_keep].copy()
        forecasts.rename(columns={'sc_cm_sb_main':'main_mean_ln'}, inplace=True)
        print('Here is the filtered and renamed dataframe for the fatalities001 model:')
        
if model == 'fatalities002':
        columns_to_keep=['country_id','month_id','main_mean', 'main_mean_ln']
        forecasts = forecasts[columns_to_keep].copy()
        print('Here is the filtered dataframe for the fatalities002 model:')

forecasts

Here is the filtered dataframe for the fatalities002 model:


,country_id,month_id,main_mean,main_mean_ln
0,1,535,0.0085,0.0085
1,2,535,0.0109,0.0108
2,3,535,0.0113,0.0112
3,4,535,0.1129,0.1070
4,5,535,0.0194,0.0193
...,...,...,...,...
6871,242,570,1.0187,0.7024
6872,243,570,0.4594,0.3781
6873,244,570,0.7030,0.5324
6874,245,570,298.6518,5.7026


In [9]:
# Add relevant transforms of the main output column

forecasts['main_mean_ln2'] = np.log1p(forecasts['main_mean_ln']).astype(float)
forecasts['main_mean'] = np.expm1(forecasts['main_mean_ln'])
print('Added a column for the linear and ln2 versions of main_mean_ln. See here:')
forecasts

Added a column for the linear and ln2 versions of main_mean_ln. See here:


,country_id,month_id,main_mean,main_mean_ln,main_mean_ln2
0,1,535,0.008536,0.0085,0.008464
1,2,535,0.010859,0.0108,0.010742
2,3,535,0.011263,0.0112,0.011138
3,4,535,0.112934,0.1070,0.101654
4,5,535,0.019487,0.0193,0.019116
...,...,...,...,...,...
6871,242,570,1.018592,0.7024,0.532039
6872,243,570,0.459509,0.3781,0.320706
6873,244,570,0.703015,0.5324,0.426835
6874,245,570,298.645467,5.7026,1.902496


In [10]:
# Generate month to plot based on first month of selected dataset and the step chosen above

t = (forecasts['month_id'].min())-1
month_to_plot = t + step 
month_to_plot

546

## Set output folder

In [11]:
import os
home = os.path.expanduser("~")
desktop = home+'/Desktop/' # (Toggle on to save to desktop) Change to your directory
#forecasts.to_csv(desktop+'forecasts.csv', index=False)

## Fetch geometries for plotting and merge with dataframe

In [12]:
engine = sa.create_engine(source_db_path) 
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, name, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)
gdf_c = gdf_ci_master.copy()


In [13]:
gdf_c

,country_id,name,in_africa,in_me,geom
0,236,Tanzania,1,0,"MULTIPOLYGON (((34.13636 -9.57117, 34.07444 -9..."
1,65,Russia,0,0,"MULTIPOLYGON (((104.26255 77.68518, 104.33638 ..."
2,39,United Kingdom,0,0,"MULTIPOLYGON (((-4.86416 54.62722, -4.9225 54...."
3,189,USSR,0,0,"MULTIPOLYGON (((104.26255 77.68518, 104.33638 ..."
4,85,Italy,0,0,"MULTIPOLYGON (((12.75357 43.971, 12.76472 43.9..."
...,...,...,...,...,...
215,133,Afghanistan,0,0,"MULTIPOLYGON (((61.27655 35.60725, 61.29638 35..."
216,187,Czechoslovakia,0,0,"MULTIPOLYGON (((14.73972 50.82388, 14.77153 50..."
217,227,Serbia and Montenegro,0,0,"MULTIPOLYGON (((18.45555 42.56583, 18.46389 42..."
218,230,Serbia,0,0,"MULTIPOLYGON (((20.08568 42.54815, 20.10469 42..."


In [14]:
data = pd.merge(forecasts, gdf_c, on="country_id", how="left")
data

,country_id,month_id,main_mean,main_mean_ln,main_mean_ln2,name,in_africa,in_me,geom
0,1,535,0.008536,0.0085,0.008464,Guyana,0,0,"MULTIPOLYGON (((-58.17262 6.81222, -58.15494 6..."
1,2,535,0.010859,0.0108,0.010742,Suriname,0,0,"MULTIPOLYGON (((-55.12796 5.82217, -55.10445 5..."
2,3,535,0.011263,0.0112,0.011138,Trinidad and Tobago,0,0,"MULTIPOLYGON (((-61.07945 10.82416, -61.07556 ..."
3,4,535,0.112934,0.1070,0.101654,Venezuela,0,0,"MULTIPOLYGON (((-66.31029 10.62602, -66.28309 ..."
4,5,535,0.019487,0.0193,0.019116,Samoa,0,0,"MULTIPOLYGON (((-172.5965 -13.50911, -172.5519..."
...,...,...,...,...,...,...,...,...,...
6871,242,570,1.018592,0.7024,0.532039,Tanzania,1,0,"MULTIPOLYGON (((34.13636 -9.57117, 34.07444 -9..."
6872,243,570,0.459509,0.3781,0.320706,Morocco,1,0,"MULTIPOLYGON (((-4.42042 35.15125, -4.35792 35..."
6873,244,570,0.703015,0.5324,0.426835,Mauritania,1,0,"MULTIPOLYGON (((-10.71639 15.4389, -10.71945 1..."
6874,245,570,298.645467,5.7026,1.902496,Sudan,1,0,"MULTIPOLYGON (((34.09223 9.47747, 33.90162 9.4..."


In [15]:
gdf = gpd.GeoDataFrame(data, geometry="geom")
gdf

,country_id,month_id,main_mean,main_mean_ln,main_mean_ln2,name,in_africa,in_me,geom
0,1,535,0.008536,0.0085,0.008464,Guyana,0,0,"MULTIPOLYGON (((-58.17262 6.81222, -58.15494 6..."
1,2,535,0.010859,0.0108,0.010742,Suriname,0,0,"MULTIPOLYGON (((-55.12796 5.82217, -55.10445 5..."
2,3,535,0.011263,0.0112,0.011138,Trinidad and Tobago,0,0,"MULTIPOLYGON (((-61.07945 10.82416, -61.07556 ..."
3,4,535,0.112934,0.1070,0.101654,Venezuela,0,0,"MULTIPOLYGON (((-66.31029 10.62602, -66.28309 ..."
4,5,535,0.019487,0.0193,0.019116,Samoa,0,0,"MULTIPOLYGON (((-172.5965 -13.50911, -172.5519..."
...,...,...,...,...,...,...,...,...,...
6871,242,570,1.018592,0.7024,0.532039,Tanzania,1,0,"MULTIPOLYGON (((34.13636 -9.57117, 34.07444 -9..."
6872,243,570,0.459509,0.3781,0.320706,Morocco,1,0,"MULTIPOLYGON (((-4.42042 35.15125, -4.35792 35..."
6873,244,570,0.703015,0.5324,0.426835,Mauritania,1,0,"MULTIPOLYGON (((-10.71639 15.4389, -10.71945 1..."
6874,245,570,298.645467,5.7026,1.902496,Sudan,1,0,"MULTIPOLYGON (((34.09223 9.47747, 33.90162 9.4..."


## Plot settings

### Overwrite relevant Mapper2 functions for flexibility in settings

In [16]:
from views_mapper2.mapper2 import Mapper2

def custom_add_colorbar(
    self,
    cmap,
    vmin,
    vmax,
    location="right",
    size="5%",
    pad=0.1,
    transparency=None,
    labelsize=10,
    tickparams=None,
):
    norm = plt.Normalize(vmin, vmax)
    if isinstance(cmap, str):
        cmap = plt.get_cmap(cmap)
    cmap = color.force_alpha_colormap(cmap=cmap, alpha=transparency)
    scalar_to_rgba = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    divider = make_axes_locatable(self.ax)
    self.cax = divider.append_axes(location, size, pad)
    self.cax.tick_params(labelsize=labelsize)

    tickvalues = (
        list(tickparams.values()) if tickparams is not None else None
    )
    self.cbar = plt.colorbar(
        scalar_to_rgba, cax=self.cax, ticks=tickvalues
    )
    if tickparams is not None:
        self.cbar.set_ticklabels(list(tickparams.keys()))
    return self

# Monkey patch the method
Mapper2.add_colorbar = custom_add_colorbar




### Set labels for plotted months in the file paths

In [17]:

if month_to_plot == 546:
    month_in_filepath = '546(June2025)'

if month_to_plot == 528:
    month_in_filepath = '528(Dec2023)'

if month_to_plot == 516:
    month_in_filepath = '516(Dec2022)'

month_in_filepath


'546(June2025)'

### Plot and save the data

In [18]:
filtered_gdf = gdf[gdf['month_id'] == month_to_plot]  

region = 'ame' #options are 'ame', 'me' or 'globe'

cmap = ['rainbow', 'binary']

for cmaps in cmap:
    test_map = Mapper2(
        width=10,   # dimension width
        height=10,  # dimension height
        frame_on=True,
        bbox=bbox_from_cid(f'{region}'), 
    ).add_layer(
        gdf=filtered_gdf,  
        cmap=cmaps,
        transparency=1,
        background=None, 
        map_dictionary=dictionary_stand_1p_10k, 
        edgecolor="black",  # border color choice
        linewidth=0.2,  # line size choice
        column="main_mean_ln", 
    )
    
    # Jim procedure for country border creation using the geometries inherent to views3
    ax = test_map.ax
    gdf_c.plot(ax=ax, edgecolor='black', linewidth=0.1, facecolor='None')

    # Print map status
    print(f'{cmaps} map all done!')

    # Save the map with appropriate filename
    test_map.save(f'{desktop}/map_{dataset}_{loa}_s{step}_for_m{month_in_filepath}_{region}_{cmaps}')

rainbow map all done!
binary map all done!
